In [7]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
wine = load_wine()
X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import joblib, matplotlib.pyplot as plt, os
def train_evaluate_save(model, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='macro'),
        'recall': recall_score(y_test, y_pred, average='macro'),
        'f1': f1_score(y_test, y_pred, average='macro')
    }
    os.makedirs('../models', exist_ok=True)
    joblib.dump(model, f'../models/{model_name}.pkl')
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=wine.target_names)
    disp.plot()
    plt.savefig(f'../results/{model_name}_confusion_matrix.png')
    plt.close()
    return metrics


In [10]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr_metrics = train_evaluate_save(lr, 'logistic_regression')

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_metrics = train_evaluate_save(rf, 'random_forest')

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state=42)
svm_metrics = train_evaluate_save(svm, 'svm')


In [13]:
import pandas as pd
metrics_df = pd.DataFrame([lr_metrics, rf_metrics, svm_metrics], index=['Logistic Regression', 'Random Forest', 'SVM'])
metrics_df.to_csv('../results/model_comparison.csv')


In [15]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment("Wine Classification")
mlflow.set_tracking_uri("http://localhost:5000")

2025/09/18 12:38:45 INFO mlflow.tracking.fluent: Experiment with name 'Wine Classification' does not exist. Creating a new experiment.


In [19]:
def train_log_mlflow(model, model_name, params, experiment_name="Wine_Classification"):
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        metrics = train_evaluate_save(model, model_name)
        for metric, value in metrics.items():
            mlflow.log_metric(metric, value)
        mlflow.log_artifact(f'../results/{model_name}_confusion_matrix.png')
        mlflow.sklearn.log_model(model, "model")
        return mlflow.active_run().info.run_id


In [ ]:
lr_params1 = {'max_iter': 1000, 'C': 0.1}
lr_run1_id = train_log_mlflow(LogisticRegression(**lr_params1), 'lr_run1', lr_params1)  
lr_params2 = {'max_iter': 1000, 'C': 1.0}
lr_run2_id = train_log_mlflow(LogisticRegression(**lr_params2), 'lr_run2', lr_params2)  
lr_params3 = {'max_iter': 1000, 'C': 10.0}
lr_run3_id = train_log_mlflow(LogisticRegression(**lr_params3), 'lr_run3', lr_params3)  


2025/09/18 12:40:59 INFO mlflow.tracking.fluent: Experiment with name 'Wine_Classification' does not exist. Creating a new experiment.
2025/09/18 12:41:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:41:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run lr_run1 at: http://localhost:5000/#/experiments/762586276057879111/runs/0d9d6107996e4885a43d158f25ed641e.
2025/09/18 12:41:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/762586276057879111.
2025/09/18 12:41:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:41:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run lr_run2 at: http://localhost:5000/#/experiments/762586276057879111

In [21]:
rf_params1 = {'n_estimators': 50, 'random_state': 42}
rf_run1_id = train_log_mlflow(RandomForestClassifier(**rf_params1), 'rf_run1', rf_params1)  
rf_params2 = {'n_estimators': 100, 'random_state': 42}
rf_run2_id = train_log_mlflow(RandomForestClassifier(**rf_params2), 'rf_run2', rf_params2)  
rf_params3 = {'n_estimators': 200, 'random_state': 42}
rf_run3_id = train_log_mlflow(RandomForestClassifier(**rf_params3), 'rf_run3', rf_params3)  


2025/09/18 12:44:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:44:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run rf_run1 at: http://localhost:5000/#/experiments/762586276057879111/runs/55fdf0cdd9e04957ad6329d2900cba95.
2025/09/18 12:44:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/762586276057879111.
2025/09/18 12:44:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:44:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run rf_run2 at: http://localhost:5000/#/experiments/762586276057879111/runs/6180c404083345f6bf636fd74d4fc553.
2025/09/18 12:44:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http:/

In [22]:
svm_params1 = {'kernel': 'linear', 'C': 0.1, 'random_state': 42}
svm_run1_id = train_log_mlflow(SVC(**svm_params1), 'svm_run1', svm_params1)  
svm_params2 = {'kernel': 'linear', 'C': 1.0, 'random_state': 42}
svm_run2_id = train_log_mlflow(SVC(**svm_params2), 'svm_run2', svm_params2)  
svm_params3 = {'kernel': 'linear', 'C': 10.0, 'random_state': 42}
svm_run3_id = train_log_mlflow(SVC(**svm_params3), 'svm_run3', svm_params3)  


2025/09/18 12:45:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:45:54 INFO mlflow.tracking._tracking_service.client: 🏃 View run svm_run1 at: http://localhost:5000/#/experiments/762586276057879111/runs/baa98ca88e3b4760ac7fe94511e3622b.
2025/09/18 12:45:54 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/762586276057879111.
2025/09/18 12:45:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/09/18 12:45:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run svm_run2 at: http://localhost:5000/#/experiments/762586276057879111/runs/5494cc337e74468281be83377069a4fa.
2025/09/18 12:45:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http

In [23]:
run_id = lr_run2_id
mlflow.register_model(f"runs:/{run_id}/model", "WineClassifier")

Successfully registered model 'WineClassifier'.
2025/09/18 13:01:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: WineClassifier, version 1
Created version '1' of model 'WineClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1758182464702, current_stage='None', description='', last_updated_timestamp=1758182464702, name='WineClassifier', run_id='2d22b7f14a544422b2b6d2ac9d2ae80c', run_link='', source='mlflow-artifacts:/762586276057879111/2d22b7f14a544422b2b6d2ac9d2ae80c/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>